# Hatodik gyakorlat - LP Solverek (PuLP)

In [3]:
import pulp as pl
import numpy as np
import matplotlib.pyplot as plt
import itertools as it

# Modellezés PuLP-pal

Modell létrehozása PuLP-pal:

In [2]:
prob = pl.LpProblem(name="ProblemName",
                    sense=pl.LpMaximize) # vagy pl.LpMinimize (ez a default)

Változók létrehozása:

In [3]:
x = pl.LpVariable(name="x",
                  lowBound=1,
                  upBound=10,
                  cat=pl.LpInteger) # vagy pl.LpContinuous vagy pl.LpBinary

Változók létrehozás dictionary-ként:

In [6]:
idx = ["I1", "I2", "A1", "A2"]
y = pl.LpVariable.dicts(name="y",
                        indices=idx,
                        cat=pl.LpBinary)

Korlátok hozzáadása a modellhez:

In [7]:
prob += x + y["I1"] <= 2, "elso constraint"
prob += pl.lpSum(y) == 3, "masodik constraint"
prob += x + y["A1"] >= 3

Célfüggvény hozzáadása a modellhez:

In [8]:
cost = {
    "I1" : 5,
    "I2" : 2,
    "A1" : 3,
    "A2" : 4
}
prob += pl.lpSum(y[i] * cost[i] for i in idx)

Modell kiírása:

In [9]:
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

_C1: x + y_A1 >= 3

VARIABLES
1 <= x <= 10 Integer
0 <= y_A1 <= 1 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Modell megoldása:

In [10]:
prob.solve()

1

Probléma állapotok:

In [11]:
pl.LpStatus

{0: 'Not Solved',
 1: 'Optimal',
 -1: 'Infeasible',
 -2: 'Unbounded',
 -3: 'Undefined'}

Megoldás "megszerzése":

In [12]:
print("x=",pl.value(x))
for i in idx:
    print(y[i].name, "=", pl.value(y[i]))

x= 2.0
y_I1 = 0.0
y_I2 = 1.0
y_A1 = 1.0
y_A2 = 1.0


Célfüggvényérték:

In [13]:
pl.value(prob.objective)

9.0

Korlátok ellenőrzése:

Azt mutatja, hogy például egy $ax\leq b$ korlát esetén mennyi az $ax-b$ értéke. Ezeknek egyenlőség esetén nyilván $0$-nak kell lenni.

In [14]:
print(pl.value(prob.constraints["_C1"]))
print(pl.value(prob.constraints["elso_constraint"]))
print(pl.value(prob.constraints["masodik_constraint"]))

0.0
0.0
0.0


Korlát törlése:

In [15]:
del prob.constraints["_C1"]
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

VARIABLES
1 <= x <= 10 Integer
0 <= y_A1 <= 1 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Változó korlátok módosítása:

In [16]:
x.lowBound=2
x.upBound=4
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

VARIABLES
2 <= x <= 4 Integer
0 <= y_A1 <= 1 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



Változó típusának módosítása:

In [17]:
y["A1"].cat = pl.LpInteger
y["A1"].lowBound = 0
y["A1"].upBound = 3
print(prob)

ProblemName:
MAXIMIZE
3*y_A1 + 4*y_A2 + 5*y_I1 + 2*y_I2 + 0
SUBJECT TO
elso_constraint: x + y_I1 <= 2

masodik_constraint: y_A1 + y_A2 + y_I1 + y_I2 = 3

VARIABLES
2 <= x <= 4 Integer
0 <= y_A1 <= 3 Integer
0 <= y_A2 <= 1 Integer
0 <= y_I1 <= 1 Integer
0 <= y_I2 <= 1 Integer



# Feladatok

## Keverési feladat

Szeretnénk kikeverni $25$ tonna speciális összetételű acélt különböző acéltömbökből. A kapott keveréknek az $5\%$-a legyen szén és $5\%$-a molibdén. Négy féle acél áll rendelkezésünkre, mindegyiket tömb formájában lehet megvenni, és mindegyikből legfeljebb $1$-et. A tömbök összetétele:

| Tömb | Súly (tonna) | Szén (%) | Molibdén (%) | Ár (\$/tonna) |
| ---- | ------------ | -------- | ------------ | ------------- |
| 1    | 5            | 5        | 3            | 350           |
| 2    | 3            | 4        | 3            | 330           |
| 3    | 4            | 5        | 4            | 310           |
| 4    | 6            | 3        | 4            | 280           |

Ezen kívül megvásárolható $3$ féle ötvözet is, illetve hulladékfémet is tudunk venni. Ezekből törtmennyiségeket is lehet vásárolni. Az összetételük:

| Ötvözet  | Szén (%) | Molibdén (%) | Ár (\$/tonna) |
| -------- | -------- | ------------ | ------------- |
| 1        | 8        | 6            | 500           |
| 2        | 7        | 7            | 450           |
| 3        | 6        | 8            | 400           |
| Hulladék | 3        | 9            | 100           |

A cél a lehető legolcsóbban előállítani a kívánt ötvözetet.

In [ ]:
# problem data
# I1, ..., I4 : ingots (tömbök)
# A1, A2, A3  : alloys (ötvözetek)
# S           : scrap (hulladék)
carbon = {
    "I1" : 0.05,
    "I2" : 0.04,
    "I3" : 0.05,
    "I4" : 0.03,
    "A1" : 0.08,
    "A2" : 0.07,
    "A3" : 0.06,
    "S"  : 0.03
}

molybdenium = {
    "I1" : 0.03,
    "I2" : 0.03,
    "I3" : 0.04,
    "I4" : 0.04,
    "A1" : 0.06,
    "A2" : 0.07,
    "A3" : 0.08,
    "S"  : 0.09
}

weight = {
    "I1" : 5,
    "I2" : 3,
    "I3" : 4,
    "I4" : 6
}


cost = {
    "I1" : 350,
    "I2" : 330,
    "I3" : 310,
    "I4" : 280,
    "A1" : 500,
    "A2" : 450,
    "A3" : 400,
    "S"  : 100
}

## Sudoku
Készítsd el a Sudoku feladat IP modelljét és oldd meg! A megoldást vizualizáld! 

__Bónusz:__ oldd meg a világ legnehezebb sudokuját: https://abcnews.go.com/blogs/headlines/2012/06/can-you-solve-the-hardest-ever-sudoku

## Legkisebb abszolút eltérés

Adottak $(x_1,y_1),(x_2,y_2),\dots,(x_n,y_n)$ pontok, és szeretnék rájuk illeszteni egy egyenest, amelynek a lehető legkisebb az abszolút eltérése a pontoktól. Vagyis keressük azt az $a$ és $b$ paramétert, ami minimalizálja a

$$
    \sum_{i=1}^n \left|a x_i+b-y_i\right|
$$

összeget. 
- Modellezd LP-ként!
- Generálj random pontokat (például vegyél egy egyenest, azon véletlen pontokat és told el őket valamekkora szimmetrikus errorral) és oldd meg a feladatot!
- Ábrázold a kapott regressziós egyenest és a pontokat!

_Hint: nyilván valami trükköt kell alkalmazni, mivel az abszolutérték függvény nem lineáris._

## Bútorlap-szabászat

Tegyük fel, hogy van egy $W\times H$ méretű téglalap alapú bútorlapom, és $n$ darab kisebb darabot szeretnék kivágni belőle. Az $i$. darab $w_i\times h_i$ méretű téglalap. Készíts el egy modellt, amellyel el lehet dönteni, hogy ki tudom-e vágni a bútorlapokat.

Legyen most $W=3\mathrm{m}$, $H=4\mathrm{m}$ és legyen 4 darab, amit ki akarunk vágni:

| Munkadarab | Szélesség (m) | Magasság (m) |
| ---------- | ------------- | ------------ |
| i1         | 1             | 3            |
| i2         | 2             | 1            |
| i3         | 2             | 2            |
| i4         | 3             | 1            |

Ki tudjuk-e vágni őket a bútorlapból, és ha igen, hogyan?
__Bónusz:__ rajzold is ki az elrendezést!